In [93]:
import pandas as pd
import numpy as np
import json
import math

## Load New Name

In [94]:
new_names = pd.read_csv("codeAndName v2.csv")
new_names.head()

,New article name,Article,Description
0,00GovStr,00TRDBDKSMJ,"Redudant Governance Structures - Rowley, T; Be..."
1,00LatCon,00ELDKQQ,Spreading and shifting costs of lateral contro...
2,00WrkGrp,00PJHKMCDKODHDP,"Choosing Work Group Members; Hinds, PJ; Carley..."
3,01PreTran,01PDDKJMS,Pre-transitive balance mechanisms for signed n...
4,02Festsch,02NEFDKSN,Festschrift for Linton C Freeman: Introduction...


In [95]:
newInfo = list(zip(new_names['New article name'],new_names['Description']))
oldID = list(new_names['Article'])

# dictionary that look up the old article id for the new id and description
article_dict = dict(zip(new_names['Article'],new_names['New article name']))

# dictionary to look up the discription with new artical name
descrptn_dict = dict(zip(new_names['New article name'],new_names['Description']))

# Change data into one mode and two mode

## Load data

In [96]:
df = pd.read_excel("Krackhardt Web of Science Data v3.xlsx")
df.head()

,Article,Description,Authors,Author Full Name,Document Title,Publication Name,Publication year
0,00TRDBDKSMJ,"Redudant Governance Structures - Rowley, T; Be...","Adler, PS; Kwon, SW","Adler, PS; Kwon, SW",Social capital: Prospects for a new concept,ACADEMY OF MANAGEMENT REVIEW,2002
1,00TRDBDKSMJ,NaN,"Ahuja, G","Ahuja, G","Collaboration networks, structural holes, and ...",ADMINISTRATIVE SCIENCE QUARTERLY,2000
2,00TRDBDKSMJ,NaN,"Dyer, JH; Nobeoka, K","Dyer, JH; Nobeoka, K",Creating and managing a high-performance knowl...,STRATEGIC MANAGEMENT JOURNAL,2000
3,00TRDBDKSMJ,NaN,"Inkpen, AC; Tsang, EWK","Inkpen, AC; Tsang, EWK","Social capital, networks, and knowledge transfer",ACADEMY OF MANAGEMENT REVIEW,2005
4,00TRDBDKSMJ,NaN,"Baum, JAC; Calabrese, T; Silverman, BS","Baum, JAC; Calabrese, T; Silverman, BS",Don't go it alone: Alliance network compositio...,STRATEGIC MANAGEMENT JOURNAL,2000


## Abnormal data detection

In [97]:
#Multi-records of the same Article-citation pair

article_citation = df.drop("Description", axis = 1).groupby(["Article", "Document Title"]).count()

multi_citation = article_citation[(article_citation["Authors"] > 1) 
                                 | (article_citation["Author Full Name"] > 1)
                                 | (article_citation["Publication Name"] > 1)
                                 | (article_citation["Publication year"] > 1)]

        
multi_citation = multi_citation.index.values.tolist()
multi_citation

[('00TRDBDKSMJ',
  'Capacity Sharing Issue in an Electronic Co-Opetitive Network: A Simulative Approach'),
 ('00TRDBDKSMJ',
  "Collaboration partner portfolio along the growth of Chinese firms' innovation capability: configuration, evolution and pattern"),
 ('00TRDBDKSMJ',
  'Joint R&D projects: Experiences in the context of European technology policy'),
 ('00TRDBDKSMJ',
  'Network embeddedness and the exploration of novel technologies: Technological distance, betweenness centrality and density'),
 ('00TRDBDKSMJ',
  'With a Little Help from Our Colleagues: A Longitudinal Study of Social Networks for Innovation'),
 ('06SPBKMCDKSN', 'The Who of Systemic Thinking'),
 ('88DKSN',
  'Win Friends and Influence People Relationships as Conduits of Organizational Culture in Temporary Placement Agencies'),
 ('92DKNO:SFA',
  "A Social Capital Perspective on IT Professionals' Work Behavior and Attitude")]

In [98]:
#Multi-description of the same article

article_description = df.dropna(subset=["Description"])[["Article", "Description"]].groupby(["Article"]).count()

multi_description = article_description[article_description["Description"] > 1]
        
multi_description = multi_description.index.values.tolist()
multi_description

[]

## Two mode

In [99]:
rows = df['Document Title'].unique()
cols = df['Article'].unique()
two_mode = pd.DataFrame(data=np.zeros((rows.shape[0],cols.shape[0]),dtype=int),index=rows,columns=cols)

row_dict = dict(zip(rows,range(rows.shape[0])))
col_dict = dict(zip(cols,range(cols.shape[0])))

In [100]:
for ind in df.index:
    article_name = df['Document Title'][ind]
    doc_name = df['Article'][ind]
    two_mode.at[article_name, doc_name] += 1
    
two_mode.head()

,00TRDBDKSMJ,92DKNO:SFA,90DKASQ,88DKSRNSPQ,88DKSN,93DKJRHHBR,87DKSN,94MKDKAMJ,87DKSN1,07DDDKTSP,...,96JBCMDKGD,15PPAPJLDKEMJ,14KGDKLWSMR,95DKASQ,77LLDKHO,14PPJLDKPEC,14DKCPOSN,07DKTSJRSSSASS,02NEFDKSN,98PHFHDKCA
Social capital: Prospects for a new concept,1,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"Collaboration networks, structural holes, and innovation: A longitudinal study",1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Creating and managing a high-performance knowledge-sharing network: The Toyota case,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"Social capital, networks, and knowledge transfer",1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Don't go it alone: Alliance network composition and startups' performance in Canadian biotechnology,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Replace the column names to the new names

In [101]:
new_cols = [article_dict[i] for i in cols]    # change the list of column names to the new version
two_mode.columns = new_cols  # replace the column names
two_mode

,00GovStr,92Philos,90Polit,88Crises,88QAP2,93InfNet,87CSS,94Bring,87QAP1,07QAP3,...,03Vis2,15EMfam2,14Simmel4,95RevSH,77Povrty,14AdEvnt,14Accur,07DiscHand,02Festsch,98Polyn
Social capital: Prospects for a new concept,1,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"Collaboration networks, structural holes, and innovation: A longitudinal study",1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Creating and managing a high-performance knowledge-sharing network: The Toyota case,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"Social capital, networks, and knowledge transfer",1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Don't go it alone: Alliance network composition and startups' performance in Canadian biotechnology,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
POVERTY AND CULTURE - EMPIRICAL-EVIDENCE AND IMPLICATIONS FOR PUBLIC-POLICY,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
STRATEGIES FOR TEACHING GREATER TOLERANCE OF CULTURAL-DIVERSITY,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
Medication safety in emergency medical services: approaching an evidence-based method of verification to reduce errors,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
"Local dependence in random graph models: characterization, properties and statistical inference",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


## One mode

In [102]:
#using dot product of two-mode to get the number of co-citation

A = np.array(two_mode)
G = np.dot(A.T, A)
one_mode = pd.DataFrame(data=G, index=new_cols, columns=new_cols)
    
one_mode.head()

,00GovStr,92Philos,90Polit,88Crises,88QAP2,93InfNet,87CSS,94Bring,87QAP1,07QAP3,...,03Vis2,15EMfam2,14Simmel4,95RevSH,77Povrty,14AdEvnt,14Accur,07DiscHand,02Festsch,98Polyn
00GovStr,856,74,24,11,5,6,3,14,7,2,...,0,0,1,2,0,0,0,0,0,0
92Philos,74,630,50,30,28,32,25,31,18,7,...,0,0,0,0,0,0,0,0,0,0
90Polit,24,50,583,40,33,40,84,59,16,6,...,0,0,0,0,0,0,0,0,0,0
88Crises,11,30,40,420,20,28,14,17,15,9,...,0,0,0,0,0,0,0,0,0,0
88QAP2,5,28,33,20,440,12,32,25,112,102,...,0,0,0,0,0,0,0,0,1,1


## Output the data in csv format

In [103]:
two_mode.to_csv("two_mode.csv")
one_mode.to_csv("one_mode.csv")

# Create Data Script

In [104]:
#Change one-mode dataframe into source-target list

src_dst = pd.DataFrame(columns=['source', 'target', 'value'])

for i in range(len(one_mode)):
    for j in range(i + 1, len(one_mode)):
        value = one_mode.iloc[i, j]
        if(value>0):
            src_dst = src_dst.append([{'source': one_mode.columns.values[i], 
                                       'target': one_mode.columns.values[j],
                                       'value': one_mode.iloc[i, j]}], ignore_index=True, sort=False)
src_dst

,source,target,value
0,00GovStr,92Philos,74
1,00GovStr,90Polit,24
2,00GovStr,88Crises,11
3,00GovStr,88QAP2,5
4,00GovStr,93InfNet,6
...,...,...,...
360,11EMFam1,12EMEvnt,3
361,11EMFam1,16TeamInj,2
362,11EMFam1,15EMfam2,1
363,96CommBK,95RevSH,1


## Link List

In [105]:
minVal = np.min(src_dst['value'])
maxVal = np.max(src_dst['value'])
print(minVal,maxVal)

1 112


In [106]:
#Create link list

links_list = []
not_isolates = set()


indexes = pd.Index(src_dst['source']
                   .append(src_dst['target'])
                   .reset_index(drop=True).unique())

for index, link in src_dst.iterrows():
    not_isolates.add(link['source'])
    not_isolates.add(link['target'])
    record = {"source": link['source'],
              "target": link['target'],
              "value": link['value']}
    links_list.append(record)
    print('{ target: "' + link['target'] + '", source: "' 
          + link['source'] + '", strength: ' + str(1 / (1 + math.exp(-link['value']))  / 10) 
          +  " ,original: " + str(link['value']) + "},")    
    # print data script

{ target: "92Philos", source: "00GovStr", strength: 0.1 ,original: 74},
{ target: "90Polit", source: "00GovStr", strength: 0.09999999999622486 ,original: 24},
{ target: "88Crises", source: "00GovStr", strength: 0.0999983298578152 ,original: 11},
{ target: "88QAP2", source: "00GovStr", strength: 0.09933071490757153 ,original: 5},
{ target: "93InfNet", source: "00GovStr", strength: 0.09975273768433654 ,original: 6},
{ target: "87CSS", source: "00GovStr", strength: 0.09525741268224333 ,original: 3},
{ target: "94Bring", source: "00GovStr", strength: 0.09999991684719722 ,original: 14},
{ target: "87QAP1", source: "00GovStr", strength: 0.09990889488055994 ,original: 7},
{ target: "07QAP3", source: "00GovStr", strength: 0.08807970779778823 ,original: 2},
{ target: "06Cent", source: "00GovStr", strength: 0.08807970779778823 ,original: 2},
{ target: "10Activ", source: "00GovStr", strength: 0.09999999943972035 ,original: 19},
{ target: "86Snow", source: "00GovStr", strength: 0.08807970779778823

{ target: "87CSS", source: "93InfNet", strength: 0.09999999586006245 ,original: 17},
{ target: "94Bring", source: "93InfNet", strength: 0.09999999586006245 ,original: 17},
{ target: "87QAP1", source: "93InfNet", strength: 0.09990889488055994 ,original: 7},
{ target: "07QAP3", source: "93InfNet", strength: 0.09820137900379085 ,original: 4},
{ target: "06Cent", source: "93InfNet", strength: 0.09933071490757153 ,original: 5},
{ target: "10Activ", source: "93InfNet", strength: 0.09990889488055994 ,original: 7},
{ target: "86Snow", source: "93InfNet", strength: 0.09975273768433654 ,original: 6},
{ target: "00WrkGrp", source: "93InfNet", strength: 0.09996646498695336 ,original: 8},
{ target: "85FrLeave", source: "93InfNet", strength: 0.09820137900379085 ,original: 4},
{ target: "99ClsFar", source: "93InfNet", strength: 0.09975273768433654 ,original: 6},
{ target: "90Cult", source: "93InfNet", strength: 0.09996646498695336 ,original: 8},
{ target: "96Incon", source: "93InfNet", strength: 0.09

{ target: "01PreTran", source: "96HistBal", strength: 0.09933071490757153 ,original: 5},
{ target: "06Simmel4", source: "96HistBal", strength: 0.08807970779778823 ,original: 2},
{ target: "97Viscos", source: "97Vis1", strength: 0.07310585786300049 ,original: 1},
{ target: "03Vis2", source: "97Vis1", strength: 0.07310585786300049 ,original: 1},
{ target: "02Festsch", source: "97Vis1", strength: 0.07310585786300049 ,original: 1},
{ target: "03NetChng", source: "10HlthRec", strength: 0.09525741268224333 ,original: 3},
{ target: "13TeamCom", source: "10HlthRec", strength: 0.07310585786300049 ,original: 1},
{ target: "09RvJack", source: "06Simmel4", strength: 0.07310585786300049 ,original: 1},
{ target: "13TeamCom", source: "03NetChng", strength: 0.08807970779778823 ,original: 2},
{ target: "11EMFam1", source: "12TeamCon", strength: 0.09525741268224333 ,original: 3},
{ target: "15EMfam2", source: "12TeamCon", strength: 0.07310585786300049 ,original: 1},
{ target: "12EMEvnt", source: "11EMFa

In [107]:
links_list

[{'source': '00GovStr', 'target': '92Philos', 'value': 74},
 {'source': '00GovStr', 'target': '90Polit', 'value': 24},
 {'source': '00GovStr', 'target': '88Crises', 'value': 11},
 {'source': '00GovStr', 'target': '88QAP2', 'value': 5},
 {'source': '00GovStr', 'target': '93InfNet', 'value': 6},
 {'source': '00GovStr', 'target': '87CSS', 'value': 3},
 {'source': '00GovStr', 'target': '94Bring', 'value': 14},
 {'source': '00GovStr', 'target': '87QAP1', 'value': 7},
 {'source': '00GovStr', 'target': '07QAP3', 'value': 2},
 {'source': '00GovStr', 'target': '06Cent', 'value': 2},
 {'source': '00GovStr', 'target': '10Activ', 'value': 19},
 {'source': '00GovStr', 'target': '86Snow', 'value': 2},
 {'source': '00GovStr', 'target': '00WrkGrp', 'value': 1},
 {'source': '00GovStr', 'target': '85FrLeave', 'value': 1},
 {'source': '00GovStr', 'target': '99ClsFar', 'value': 2},
 {'source': '00GovStr', 'target': '90Cult', 'value': 2},
 {'source': '00GovStr', 'target': '96Incon', 'value': 3},
 {'source'

In [113]:
nodes_list = []

#article_description = df[["Article", "Description"]].groupby('Article').nth(0).reset_index()

for i in range(len(one_mode)):
    name = one_mode.columns.values.tolist()[i]
    if name in not_isolates:
        year = int(one_mode.columns.values.tolist()[i][0: 2])
        year = (1900 + year) if year > 20 else (2000 + year)
        nodes_list.append({"name": name, 
                            "group": year,
                            "size": int(one_mode.iloc[i, i]),
                            "meta_node": descrptn_dict[name]})
        print('{id: "' + name +'", label: "' + descrptn_dict[name] + '",' + 'level:' + str(year) + ' ,total_citation:'
         + str(int(one_mode.iloc[i, i])) + "}," )

{id: "00GovStr", label: "Redudant Governance Structures - Rowley, T; Behrens, D; Krackhardt, D; Strategic Management Journal - March 2000",level:2000 ,total_citation:856},
{id: "92Philos", label: "The Strength of Strong Ties - Krackhardt, D - Networks & Organizations: Structure, Form, & Action - 1992",level:1992 ,total_citation:630},
{id: "90Polit", label: "Assessing the Political Landscape - Krackhardt, D; Administrative Science Quarterly - June 1992",level:1990 ,total_citation:583},
{id: "88Crises", label: "Informal Network and Organizational Crises - Krackhardt, D; Stern, RN; Social Psychology Quarterly - June 1988",level:1988 ,total_citation:420},
{id: "88QAP2", label: "Predicting with Networks - Krackhardt, D; Social Networks - Dec. 1988",level:1988 ,total_citation:440},
{id: "93InfNet", label: "Informal Networks - Krackhardt, D; Hanson JR - Harvard Business Review - Jul/Aug 1993",level:1993 ,total_citation:385},
{id: "87CSS", label: "Cognitive Social-Structures - Krackhardt, D; S

In [111]:
nodes_list[0: 5]

[{'name': '00GovStr',
  'group': 2000,
  'size': 856,
  'meta_node': 'Redudant Governance Structures - Rowley, T; Behrens, D; Krackhardt, D; Strategic Management Journal - March 2000'},
 {'name': '92Philos',
  'group': 1992,
  'size': 630,
  'meta_node': 'The Strength of Strong Ties - Krackhardt, D - Networks & Organizations: Structure, Form, & Action - 1992'},
 {'name': '90Polit',
  'group': 1990,
  'size': 583,
  'meta_node': 'Assessing the Political Landscape - Krackhardt, D; Administrative Science Quarterly - June 1992'},
 {'name': '88Crises',
  'group': 1988,
  'size': 420,
  'meta_node': 'Informal Network and Organizational Crises - Krackhardt, D; Stern, RN; Social Psychology Quarterly - June 1988'},
 {'name': '88QAP2',
  'group': 1988,
  'size': 440,
  'meta_node': 'Predicting with Networks - Krackhardt, D; Social Networks - Dec. 1988'}]

## Output to files

In [109]:
json_prep = {"links":links_list, "nodes":nodes_list}
json_dump = json.dumps(json_prep, indent=1, sort_keys=True)

In [110]:
filename_out = 'link_node.json'
json_out = open(filename_out,'w')
json_out.write(json_dump)
json_out.close()

In [18]:
new_names = pd.read_csv("codeAndName v2.csv")
new_names.head()

,New article name,Article,Description
0,00GovStr,00TRDBDKSMJ,"Redudant Governance Structures - Rowley, T; Be..."
1,00LatCon,00ELDKQQ,Spreading and shifting costs of lateral contro...
2,00WrkGrp,00PJHKMCDKODHDP,"Choosing Work Group Members; Hinds, PJ; Carley..."
3,01PreTran,01PDDKJMS,Pre-transitive balance mechanisms for signed n...
4,02Festsch,02NEFDKSN,Festschrift for Linton C Freeman: Introduction...


In [24]:
newInfo = list(zip(new_names['New article name'],new_names['Description']))
oldID = list(new_names['Article'])

# dictionary that look up the old article id for the new id and description
nodeDict = dict(zip(oldID,newInfo))
nodeDict

{'00TRDBDKSMJ': ('00GovStr',
  'Redudant Governance Structures - Rowley, T; Behrens, D; Krackhardt, D; Strategic Management Journal - March 2000'),
 '00ELDKQQ': ('00LatCon',
  'Spreading and shifting costs of lateral control among peers - Lazega, E; Krackhardt, D - Quality & Quantity - May 2000'),
 '00PJHKMCDKODHDP': ('00WrkGrp',
  'Choosing Work Group Members; Hinds, PJ; Carley, KM; Krackhardt, D; et al.; Organizational Behavior and Human Decision Processes - Mar 2000'),
 '01PDDKJMS': ('01PreTran',
  'Pre-transitive balance mechanisms for signed networks - Doreian, P; Krackhardt, D - Journal of Mathematical Sociology - 2001'),
 '02NEFDKSN': ('02Festsch',
  'Festschrift for Linton C Freeman: Introduction - Friedkin, NE; Krackhardt, D - Social Networks - Oct 2002'),
 '02DKMKSN': ('02Simmel3',
  'Structure, culture, and Simmelian ties in entrepreneurial firms - Krackhardt, D; Kilduff, M - Social Networks - July 2002'),
 '03DDPHFDKJMS': ('03Equil',
  'An equilibrium-correction model for d

In [22]:
list(zip(new_names['New article name'],new_names['Description']))

[('00GovStr',
  'Redudant Governance Structures - Rowley, T; Behrens, D; Krackhardt, D; Strategic Management Journal - March 2000'),
 ('00LatCon',
  'Spreading and shifting costs of lateral control among peers - Lazega, E; Krackhardt, D - Quality & Quantity - May 2000'),
 ('00WrkGrp',
  'Choosing Work Group Members; Hinds, PJ; Carley, KM; Krackhardt, D; et al.; Organizational Behavior and Human Decision Processes - Mar 2000'),
 ('01PreTran',
  'Pre-transitive balance mechanisms for signed networks - Doreian, P; Krackhardt, D - Journal of Mathematical Sociology - 2001'),
 ('02Festsch',
  'Festschrift for Linton C Freeman: Introduction - Friedkin, NE; Krackhardt, D - Social Networks - Oct 2002'),
 ('02Simmel3',
  'Structure, culture, and Simmelian ties in entrepreneurial firms - Krackhardt, D; Kilduff, M - Social Networks - July 2002'),
 ('03Equil',
  'An equilibrium-correction model for dynamic network data - Dekker, D; Franses, PH; Krackhardt, D - Journal of Mathematical Sociology - Ap